## Masking

In [1]:
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import SimpleMovingAverage, AverageDollarVolume

pipelineを用いて算出する際に、特定の銘柄を無視したほうがよい場合があります。

1. 計算コストが高いため、算出対象を特定の銘柄のみとしたい場合。
   例: 回帰係数を算出するFactor([RollingLinearRegressionOfReturns](https://www.quantopian.com/help#quantopian_pipeline_factors_RollingLinearRegressionOfReturns))
2. 銘柄同士の比較を実施する際に、流動性が低い銘柄を対象外としたい場合

FactorとFactorのメソッドはキーワード引数 `mask` を設定することができます。この引数は銘柄に対した Filter である必要があります。

### Masking Factors

売買代金が10,000,000ドルを超える銘柄のみをpipelineで扱うケースを考えてみます。下記のコードでは `high_dollar_volume` という Filter を作成し、`SimpleMovingAverage()` のキーワード引数 `mask` に渡しています。これを実施することで出来高が多い銘柄のみを算出対象にできます。

In [2]:
# Dollar volume factor
dollar_volume = AverageDollarVolume(window_length=30)

# High dollar volume filter
high_dollar_volume = (dollar_volume > 10000000)

# Average close price factors
mean_close_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=10, mask=high_dollar_volume)
mean_close_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length=30, mask=high_dollar_volume)

# Relative difference factor
percent_difference = (mean_close_10 - mean_close_30) / mean_close_30

マスクを設定することで計算対象が8000銘柄程度から2000銘柄程度に抑えることができました。

### Masking Filters

マスクは `top` や `bottom` など、filterのメソットに対して指定できます。

マスクは、組み合わせる前のステップでfilterを適用する場合に便利です。たとえば、下記の3つの条件に合致させるためのコードは次のようにします。
1. 始値が最も高い上位50
2. 売買代金が上位10％
3. 終値が上位10%

In [3]:
# Dollar volume factor
dollar_volume = AverageDollarVolume(window_length=30)

# High dollar volume filter
high_dollar_volume = dollar_volume.percentile_between(90,100)

# Top open price filter (high dollar volume securities)
top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

# Top percentile close price filter (high dollar volume, top 50 open price)
high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)

上記で作成した `high_close_price` を `Pipeline()` のキーワード引数 `screen` に渡します。

In [4]:
def make_pipeline():

    # Dollar volume factor
    dollar_volume = AverageDollarVolume(window_length=30)

    # High dollar volume filter
    high_dollar_volume = dollar_volume.percentile_between(90,100)

    # Top open securities filter (high dollar volume securities)
    top_open_price = USEquityPricing.open.latest.top(50, mask=high_dollar_volume)

    # Top percentile close price filter (high dollar volume, top 50 open price)
    high_close_price = USEquityPricing.close.latest.percentile_between(90, 100, mask=top_open_price)

    return Pipeline(
        screen=high_close_price
    )

上記のpipelineを実行すると、2015年5月5日の時点では5銘柄が処理されました。

In [5]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print 'Number of securities that passed the filter: %d' % len(result)

Number of securities that passed the filter: 5


次のレッスンではclassifiersを実施します。